In [ ]:
import pandas as pd
import numpy as np
import cvxpy as cvx
%matplotlib inline  
pd.options.display.max_rows=20

In [ ]:
prices = pd.read_csv("prices.csv", index_col=0)

returns=prices.pct_change().fillna(0.0)
returns

In [ ]:
print(100*16*returns.std().sort_values())

In [ ]:
# Severe problems with data quality here...
prices["UBS Group AG"].plot()

In [ ]:
def min_var(matrix):
    w = cvx.Variable(matrix.shape[1])
    
    cvx.Problem(cvx.Minimize(cvx.norm(matrix.values*w,2)), 
                [0 <= w, cvx.sum(w) == 1]).solve()
    
    return 100*pd.Series(index=matrix.keys(), data=w.value).sort_values()

min_var(returns).map('{:,.2f}%'.format)

In [ ]:
def ridge(matrix, lamb_balance=0):
    def __objective():
        return cvx.Minimize(cvx.norm(matrix.values*w,2) + 
                            lamb_balance*cvx.norm(w,2))

    w = cvx.Variable(matrix.shape[1])
    cvx.Problem(__objective(), [0 <= w, cvx.sum(w) == 1]).solve()
    return 100*pd.Series(index=matrix.keys(), data=w.value).sort_values()

ridge(returns, 1).map('{:,.2f}%'.format)

In [ ]:
def ElasticNet(matrix, w0, lamb_balance=0, lamb_trades=0):

    def __objective():
        return cvx.Minimize(cvx.norm(matrix.values*w,2) + 
                            lamb_balance*cvx.norm(w,2) + 
                            lamb_trades*cvx.norm(w-w0,1))

    w = cvx.Variable(matrix.shape[1])
    cvx.Problem(__objective(), [0 <= w, cvx.sum(w) == 1]).solve()
    return 100*pd.Series(index=matrix.keys(), data=w.value).sort_values()

ElasticNet(returns, w0=0.05, lamb_balance=1, lamb_trades=0.03).map('{:,.2f}%'.format)

In [ ]:
def ElasticNet(matrix, w0, lamb_balance=0, lamb_trades=0):

    def __objective():
        return cvx.Minimize(cvx.norm(matrix.values*w,2) + 
                            lamb_balance*cvx.norm(w,2) + 
                            lamb_trades*cvx.norm(w-w0,1))

    w = cvx.Variable(matrix.shape[1])
    cvx.Problem(__objective(), [0 <= w, cvx.sum(w) == 1]).solve()
    return 100*pd.Series(index=matrix.keys(), data=w.value).sort_values()